## Getting a bit more deeper into Agents

### Conversational ReAct Agents
This agent is designed for use in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [7]:
from langchain.agents import create_react_agent, initialize_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')

In [4]:
tools = load_tools(["llm-math"],  llm=llm)

In [13]:


from langchain import hub
#from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent,load_tools

prompt = hub.pull("hwchase17/react")
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')
# memory should not be more than the model tokens
# keeps the results of the past conversation to use in the current conversation
memory = ConversationBufferMemory(memory_key="chat_history")
tools = load_tools(["llm-math"],  llm=llm)


agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=3, memory=memory)


#initialize agent with tools
conversational_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)


/Users/reza.ghafari/.local/share/virtualenvs/llm-orielly-GenAI-RAGApplicationLangchain-haYDCRmz/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
output_1=conversational_agent.run("Add 7 to 9 and tell me the result")
output_2=conversational_agent.run("Add 5 to the previous result")

/var/folders/yx/h4n08kfj24v8y1_6xhj4p_6c0000gq/T/ipykernel_9895/2625776580.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_1=conversational_agent.run("Add 7 to 9 and tell me the result")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I should use the Calculator tool to add 7 to 9.
Action: Calculator
Action Input: 7 + 9
Observation: Answer: 16
Thought:

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I now know the final answer
Final Answer: 16

> Finished chain.
I need to use the Calculator tool to add 5 to the previous result.
Action: Calculator
Action Input: 5 + previous result
Observation: Answer: 13.222831614237718
Thought:I now know the final answer
Final Answer: 13.222831614237718

> Finished chain.


In [56]:
print(output_1)
print(output_2)

The result of adding 7 to 9 is 16.
The result of adding 5 to the previous result of 16 is 21.


## ReAct Docstore
This agent utilizes the React framework to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A `docstore` in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [15]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [17]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")]

#intializing the agent
llm = OpenAI(temperature=0)
docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=8
)

/var/folders/yx/h4n08kfj24v8y1_6xhj4p_6c0000gq/T/ipykernel_9895/631535798.py:1: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 1.0
  docstore = DocstoreExplorer(Wikipedia())


In [18]:
docstore_agent("Tell me a few key things about Mahatma Gandhi.")

/var/folders/yx/h4n08kfj24v8y1_6xhj4p_6c0000gq/T/ipykernel_9895/836576075.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docstore_agent("Tell me a few key things about Mahatma Gandhi.")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to search Mahatma Gandhi and find key information about him.
Action: Search[Mahatma Gandhi]
Observation: Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī; 2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London and was called to the bar in June 1891, at the age of 22. After two uncertain years in India, where he was unable to start a successful law practice, Gandhi moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He went on to live in South Africa for 21

{'input': 'Tell me a few key things about Mahatma Gandhi.',
 'output': 'Indian lawyer, anti-colonial nationalist, political ethicist, nonviolent resistance, Father of the Nation, Gandhi Jayanti, International Day of Nonviolence, assassinated by militant Hindu nationalist'}

## Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions. This allows llm to give answers to a question it was not trained on.

In [19]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="useful for when you need to search the internet for information"
    )

]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [20]:
print(agent.invoke("Question: Who was the president of USA when the first Moom landing took place?"))

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 Yes.
Follow up: When did the first Moon landing take place?
Intermediate answer: In July 1969, Apollo 11 astronauts Neil A. Armstrong, Michael Collins, and Edwin E. "Buzz" Aldrin completed humanity's first landing on the Moon. They fulfilled President John F. Kennedy's national goal , set in May 1961, to land a man on the Moon and return him safely to the Earth before the end of the decade. Apollo 11, U.S. spaceflight during which commander Neil Armstrong and lunar module pilot Edwin ("Buzz") Aldrin, Jr., on July 20, 1969, became the first people to land on the Moon and walk the lunar surface. Apollo 11 was the culmination of the Apollo program and a massive national commitment by the United States to beat the Soviet Union in ... Apollo 11 Mission Highlights. Watch highlights of the Apollo 11 mission including the launch on July 16, 1969, the landing of the lunar module, Neil Armstrong's first steps on the Moon, splashdown, and more. Watch on NASA+. 20 July 1969—Astronaut Edwin E. Ald

The above code imports necessary modules from the LangChain library, including agents and language models. It sets up a conversational agent with a specific agent configuration called “self-ask-with-search”. The agent utilizes the “Intermediate Answer” tool, which performs Wikipedia searches.